In [1]:
%run Accuracy_Module.py
%run DataLoading.py
%run load_and_organize_dataset.py
%run training.py
%run checkpoint.py

In [2]:
train_data, valid_data, test_data = load_dataset(64)

In [3]:
resnet152 = models.resnet152(pretrained=True)


In [4]:
resnet152.cuda()
resnet152.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [5]:
#resetnet152
class RES_regression(nn.Module):
    def __init__(self):
        super(RES_regression, self).__init__()
        self.name = "res_regression"
        self.fc1 = nn.Linear(1000, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 1)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = x.cuda()
        x = resnet152(x)
        x = x.view(x.shape[0],-1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        x = x.squeeze(1)
        
        return x

In [ ]:
res_regression = RES_regression()
res_regression.cuda()

In [ ]:
train_regression(train_data, valid_data, res_regression, model_name = 'RES152', batch_size = 32, num_epochs = 20)

In [ ]:
client = storage.Client()
bucket_name = "aps360team12"
bucket = client.get_bucket(bucket_name)
blob_name = "RES_features_bs64_lr5e-05_epoch20"
blob = bucket.get_blob(blob_name)

In [ ]:
blob.download_to_filename('RES_pretrained_epoch20')

In [ ]:
trained_data = torch.load('RES_pretrained_epoch20')

In [ ]:
print(trained_data['epoch'])

In [ ]:
res_regression.load_state_dict(trained_data['model_state_dict'])
res_regression.eval()

In [ ]:
train_net(train_data, valid_data, res_regression, pretrained = True, checkpoint = trained_data, batch_size = 64, num_epochs = 20)

In [6]:
trained_data = load_checkpoint_from_cloud('aps360team12', 'VGG_features_bs64_lr5e-05_epoch39', 'new_temp')

In [7]:
RES_best = RES_regression()
RES_best.cuda(0)
RES_best.load_state_dict(trained_data['model_state_dict'])
RES_best.eval()

RES_regression(
  (fc1): Linear(in_features=1000, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.2)
)

In [8]:
get_off_accuracy_regression(RES_best, test_data)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-bc7af967c139>", line 1, in <module>
    get_off_accuracy_regression(RES_best, test_data)
  File "/home/jupyter/training.py", line 20, in get_off_accuracy_regression
    out = model(img)
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/module.py", line 489, in __call__
    result = self.forward(*input, **kwargs)
  File "<ipython-input-5-881631306fc4>", line 14, in forward
    x = resnet152(x)
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/module.py", line 489, in __call__
    result = self.forward(*input, **kwargs)
  File "/opt/anaconda3/lib/python3.7/site-packages/torchvision/models/resnet.py", line 157, in forward
    x = self.layer3(x)
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/module.py", line 489, i

RuntimeError: CUDA out of memory. Tried to allocate 42.25 MiB (GPU 0; 15.90 GiB total capacity; 14.82 GiB already allocated; 43.44 MiB free; 199.51 MiB cached)